# Sentiment Classification

In [344]:
# Importing the libraries
import pandas as pd
import numpy as np

In [345]:
# Load the data and combine them
df_pos = pd.read_csv('data/reviews.pos' , delimiter="\t", header=None)
df_neg = pd.read_csv('data/reviews.neg', delimiter="\t", header=None)

df_neg['label'] = 'neg'
df_pos['label'] = 'pos'

df = pd.concat([df_pos, df_neg], ignore_index=True)
df.to_csv('data/combined_reviews.csv', index=False, header=['review', 'label'])

In [346]:
data = pd.read_csv('data/combined_reviews.csv')
data = data.sample(n=1000).reset_index(drop=True)

In [347]:
data.head()

,review,label
0,"zaten oral b diş fırçası kullanıyordum, yenile...",pos
1,urun gayet guzel 20.000 km kullandim silinme v...,pos
2,"genelde primayı kullanıyoruz, ara sıra can beb...",pos
3,bundan önceki g2 telefonuma göre kat kat hızlı...,pos
4,kesinlikler süper bir film herkese tavsiye ede...,pos


In [348]:
data['label'].value_counts()

label
pos    933
neg     67
Name: count, dtype: int64

In [349]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import string

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\furka\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [350]:
# Function to clean the text
# split and print the punctiation, stopwords and the text

punctuation = string.punctuation
stopwords = stopwords.words('turkish')

print(punctuation)
print(stopwords)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['acaba', 'ama', 'aslında', 'az', 'bazı', 'belki', 'biri', 'birkaç', 'birşey', 'biz', 'bu', 'çok', 'çünkü', 'da', 'daha', 'de', 'defa', 'diye', 'eğer', 'en', 'gibi', 'hem', 'hep', 'hepsi', 'her', 'hiç', 'için', 'ile', 'ise', 'kez', 'ki', 'kim', 'mı', 'mu', 'mü', 'nasıl', 'ne', 'neden', 'nerde', 'nerede', 'nereye', 'niçin', 'niye', 'o', 'sanki', 'şey', 'siz', 'şu', 'tüm', 've', 'veya', 'ya', 'yani']


In [351]:
def clean_from_stopwords(data):
    cleaned_data = []
    for d in data:
        temp = ''
        for word in d.split():
            if word not in stopwords and not word.isnumeric():
                temp += word + ' '
        cleaned_data.append(temp.strip())
    return pd.Series(cleaned_data)

# Clean data from stopwords
data['review'] = clean_from_stopwords(data['review'])

In [352]:
data['review'][0:5]

0    zaten oral b diş fırçası kullanıyordum, yenile...
1    urun gayet guzel 20.000 km kullandim silinme v...
2    genelde primayı kullanıyoruz, ara sıra can beb...
3    bundan önceki g2 telefonuma göre kat kat hızlı...
4    kesinlikler süper bir film herkese tavsiye ede...
Name: review, dtype: object

In [353]:
# Function to clean the text from punctuation
def clean_data_from_punctuation(data):
    cleaned_data = []
    for d in data:
        temp = ''
        for word in d:
            if word not in punctuation:
                temp += word
        cleaned_data.append(temp)
    return pd.Series(cleaned_data)

# Clean data from punctuation
data['review'] = clean_data_from_punctuation(data['review'])

In [354]:
data['review'][0:5]

0    zaten oral b diş fırçası kullanıyordum yenilem...
1    urun gayet guzel 20000 km kullandim silinme vs...
2    genelde primayı kullanıyoruz ara sıra can bebe...
3    bundan önceki g2 telefonuma göre kat kat hızlı...
4    kesinlikler süper bir film herkese tavsiye ede...
Name: review, dtype: object

In [355]:
# save the cleaned data
data.to_csv('data/cleaned_reviews.csv', index=False)
data = pd.read_csv('data/cleaned_reviews.csv',sep=',',names=['review', 'label'])
print(data.head())

                                              review  label
0                                             review  label
1  zaten oral b diş fırçası kullanıyordum yenilem...    pos
2  urun gayet guzel 20000 km kullandim silinme vs...    pos
3  genelde primayı kullanıyoruz ara sıra can bebe...    pos
4  bundan önceki g2 telefonuma göre kat kat hızlı...    pos
